In [9]:
import numpy as np # used for arrays
import gym # pull the environment
import time # to get the time
import math # needed for calculations
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
# import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import pickle


In [19]:
class QLearning:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        # q_table = None
        # self.create_q_table()

        self.p_space = np.linspace(-2.4, 2.4, 10)
        self.v_space = np.linspace(-4, 4, 10)
        self.a_space = np.linspace(-.2095, .2095, 10)
        self.av_space = np.linspace(-4, 4, 10)

        self.alpha = 0.1
        self.gamma = 0.99
        self.epsilon = 1
        self.epsilon_decay_rate = 0.00001
        
        # training variables
        # self.no_of_Episodes = 100
        self.is_training = True
        self.end = False
        self.train_episode_count = 0
        self.reward_training_episodes = []

        
    def trainingEpisodes(self):
        self.env = gym.make('CartPole-v1')

        self.train_episode_count = 0
        self.reward_training_episodes = []
    

        q_table = np.zeros((len(self.p_space)+1, 
                      len(self.v_space)+1, 
                      len(self.a_space)+1, 
                      len(self.av_space)+1, 
                      self.env.action_space.n)) # init a 11x11x11x11x2 array

        while(True):
            # initialise everything
            # print(" =========== training round "+str(self.train_episode_count) + " ===========")
            current_state = self.env.reset()[0]       
            self.end = False
            rewards = 0
            
            state_p = np.digitize(current_state[0],self.p_space)
            state_v= np.digitize(current_state[1],self.v_space)
            state_a=np.digitize(current_state[2],self.a_space)
            state_av=np.digitize(current_state[3],self.av_space)

            # print(" == starting state: ==")
            # print(current_state)
            # print(" == starting index: ==")
            # print(state_index)
            
            while(not self.end and rewards<1000):
                
                randomNumber=np.random.random()
                if randomNumber < self.epsilon:
                    action = self.env.action_space.sample()
                else:
                    action = np.argmax(q_table[state_p, state_v, state_a, state_av, :])

                
                new_state, reward, self.end, _ ,_ = self.env.step(action)
                new_state_p = np.digitize(new_state[0],self.p_space)
                new_state_v= np.digitize(new_state[1],self.v_space)
                new_state_a=np.digitize(new_state[2],self.a_space)
                new_state_av=np.digitize(new_state[3],self.av_space)
                                
                
                q_table[state_p, state_v, state_a, state_av, action] = q_table[state_p, state_v, state_a, state_av, action] + self.alpha * (reward + self.gamma * np.max(q_table[new_state_p, new_state_v, new_state_a, new_state_av,:]) - q_table[state_p, state_v, state_a, state_av, action])

                current_state = new_state
                state_p = new_state_p
                state_v = new_state_v
                state_a = new_state_a
                state_av= new_state_av
                rewards += reward

            
            self.reward_training_episodes.append(rewards)
            mean_rewards = np.mean(self.reward_training_episodes[len(self.reward_training_episodes)-100:])
            

            if self.train_episode_count%100==0:
                print(f'Episode: {self.train_episode_count}: {rewards}  Epsilon: {self.epsilon:0.2f}  Mean_Rewards {mean_rewards:0.1f}')

            if mean_rewards>500:
                break

            self.epsilon = max(self.epsilon - self.epsilon_decay_rate, 0)
            self.train_episode_count+=1


        self.env.close()
        
        if self.is_training:
            f = open('cartpole.pkl','wb')
            pickle.dump(q_table, f)
            f.close()

    def simulateEpisodes(self):
        self.env = gym.make('CartPole-v1', render_mode='human')
        
        f = open('cartpole.pkl', 'rb')
        q_table = pickle.load(f)
        f.close()

        self.train_episode_count = 0
        self.reward_training_episodes = []

        while(True):
            # initialise everything
            # print(" =========== training round "+str(self.train_episode_count) + " ===========")
            current_state = self.env.reset()[0]       
            self.end = False
            rewards = 0
            
            state_p = np.digitize(current_state[0],self.p_space)
            state_v= np.digitize(current_state[1],self.v_space)
            state_a=np.digitize(current_state[2],self.a_space)
            state_av=np.digitize(current_state[3],self.av_space)

            
            while(not self.end and rewards<1000):
                
                action = np.argmax(q_table[state_p, state_v, state_a, state_av, :])

                new_state, reward, self.end, _ ,_ = self.env.step(action)
                new_state_p = np.digitize(new_state[0],self.p_space)
                new_state_v= np.digitize(new_state[1],self.v_space)
                new_state_a=np.digitize(new_state[2],self.a_space)
                new_state_av=np.digitize(new_state[3],self.av_space)
                                
                
                current_state = new_state
                state_p = new_state_p
                state_v = new_state_v
                state_a = new_state_a
                state_av= new_state_av
                rewards += reward

            
            self.reward_training_episodes.append(rewards)
            mean_rewards = np.mean(self.reward_training_episodes[len(self.reward_training_episodes)-100:])
            

            if self.train_episode_count%100==0:
                print(f'Episode: {self.train_episode_count}: {rewards}  Epsilon: {self.epsilon:0.2f}  Mean_Rewards {mean_rewards:0.1f}')

            if mean_rewards>200:
                break

            self.train_episode_count+=1


        self.env.close()
        


        

        

In [20]:
a = QLearning()

a.simulateEpisodes()

/home/jdgoh/.conda/envs/pygame/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0: 609.0  Epsilon: 1.00  Mean_Rewards 609.0
